In [4]:
import pickle
import numpy as np

with open("./pickled_data/all_dfs.dat", "rb") as f:
    all_dfs = pickle.load(f)
    
#unpack all_dfs
df_final, df_95, df_96, df_97, df_98, df_99, df_100, df_101, df_102, df_103,df_104, df_105, df_106, df_107, df_108, df_109, df_110,  df_111 = all_dfs

all_dfs_complete = all_dfs[0]
all_dfs = all_dfs[1:]

### Classifier

In [8]:
class crawler_sol(object):
    def grid_search_func(self, param_grid, the_mode_in, the_vec_in, the_lab_in):
        from sklearn.model_selection import GridSearchCV

        grid_search = GridSearchCV(the_mode_in, param_grid=param_grid, cv=5)
        best_model = grid_search.fit(the_vec_in, the_lab_in)
        max_score = grid_search.best_score_
        best_params = grid_search.best_params_

        return best_model, max_score, best_params

    def model_tune(self, xform_in, label_in, param_grid_in, path_in, df):
        from sklearn.svm import SVC
        import pickle
        
        congress = df['congress'].unique()[0]

        clf_pca = SVC(probability = True)

        gridsearch_model, best, opt_params = self.grid_search_func(
        param_grid_in, clf_pca, xform_in, label_in)

        print ("Best CV performance: " + str(best))
        print (opt_params)

        #if you want more than just accuracy (recall, F1, etc.) --- do here

        clf_pca_out = SVC(probability = True)
        clf_pca_out.set_params(**gridsearch_model.best_params_)
        clf_pca_out.fit(xform_in, label_in)

        pickle.dump(clf_pca_out, open(path_in + str(congress) + 'model.pkl', 'wb'))        

        return clf_pca_out

In [25]:
def classifier(df):
    import pickle
    import pandas as pd
    import numpy as np    
    from imblearn.combine import SMOTETomek
    from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

    congress = df['congress'].unique()[0]
    print(congress)
    
    df = df[(df.speech_length >= np.percentile(df.speech_length, 20)) & (df.speech_length <= np.percentile(df.speech_length, 80))]
    
    #Define label column
    label_col = 'ideo_vec_ext'
    
    df = df[df[label_col]!="NA"]
    df[label_col] = df[label_col].astype(int) 
    #comment out to include moderates (only relevant for ideo_vec and ideo_vec_ext cols)
    df = df[df[label_col]!=0] 

    #Define text body column
    body_text = np.array(df['body_lem'])
    label = np.array(df[label_col])
    
    
    
    
#     #Define body_col and label to classify with
#     body_text = np.array(df['body_lem'])
#     label = np.array(df['ideo_vec_ext'])
      
    
#     df = df[(df.speech_length >= np.percentile(df.speech_length, 20)) & (df.speech_length <= np.percentile(df.speech_length, 80))]
#     #Define label
#     label = np.array(df['ideo_vec_ext'])
#     df[label] = df[label].astype(int) 
#     #comment out to include moderates (only relevant for ideo_vec and ideo_vec_ext cols)
#     df = df[df[label]!=0] 
    
#     body_text = np.array(df['body_lem'])
    #    df = df[df[label]!="NA"]
    
#     df[label] = df[label].astype(int) 
#     #comment out to include moderates (only relevant for ideo_vec and ideo_vec_ext cols)
#     df = df[df[label]!=0] 
    
    
    sme = SMOTETomek(random_state=42,ratio='minority')

    my_vec_tfidf_out = TfidfVectorizer()
    my_xform_tfidf_out = my_vec_tfidf_out.fit_transform(body_text)
    my_pd = pd.DataFrame(my_xform_tfidf_out.toarray())
    my_pd.columns = my_vec_tfidf_out.get_feature_names()
    X = my_pd.values

    print(pd.Series(label).value_counts())

    #balance data
    X_res, y_res = sme.fit_sample(X, label) 
    print(pd.Series(y_res).value_counts())


    my_pd = pd.DataFrame(X_res)
    my_pd.columns = my_vec_tfidf_out.get_feature_names()
    X_res = my_pd.values
    
    path_in = './vecs+pca/'
    pickle.dump(my_vec_tfidf_out, open(path_in + str(congress) + 'tfidfvectorizer.pkl', 'wb'))

    print('start svd')
    from sklearn import decomposition
    from sklearn.decomposition import TruncatedSVD
    n_comp = 1
    var_fig = 0.0
    while var_fig <= 0.95:
        svd = decomposition.TruncatedSVD(n_components=n_comp*10, algorithm='arpack')
        my_dim = svd.fit_transform(X_res)
        var_fig = svd.explained_variance_ratio_.sum()
        print(var_fig)
        n_comp += 50

    my_func = crawler_sol()

    #grid params
    print('start grim params')
    param_grid = {"gamma": ["scale", "auto"],
                  "decision_function_shape": ["ovo", "ovr"],
                 "C": [1.0,2.0]}

    the_path="./model/"

    #model tuning, train, text, validation
    the_model = my_func.model_tune(
            my_dim, y_res, param_grid, the_path, df)


    path_in = './vecs+pca/'

    congress = df['congress'].unique()[0]

    pickle.dump(svd, open(path_in + str(congress) + 'svd.pkl', 'wb'))
    print('finish')

In [26]:
# %%time
# for i, df in enumerate(all_dfs):
#     classifier(all_dfs[i])

In [27]:
classifier(df_111)

111
-1    1202
 1     505
dtype: int64
 1    1200
-1    1200
dtype: int64
start svd
0.2261407380870616
0.8181537134835816
0.9636605223628232
start grim params
Best CV performance: 0.8704166666666666
{'C': 2.0, 'decision_function_shape': 'ovo', 'gamma': 'scale'}
finish
